In [199]:
#%matplotlib inline
import sys
import os


import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndi
import scipy.stats as stt
import pandas as pd
import networkx as nx
#optional for nicer plots
import seaborn
clrs = seaborn.color_palette(n_colors=6)
seaborn.set(style='ticks',font_scale=2)



from sklearn.manifold import MDS
from sklearn.svm import SVC
from sklearn.model_selection import LeavePOut

# UPDATE THE PATH BELOW TO THE CODE FOLDER

In [200]:
sys.path.append("/Users/yves/Desktop/retreat_data_dir/code/")
from mecll.task import plot_activity_on_graph

# Load stuff

In [201]:
#all_across_task_plane_ccs

In [396]:
#These are the times (in units of the behaviour system bin running @1000Hz) at which spikes occurred

selected_session = 1

all_data_dir = '/Users/yves/Desktop/retreat_data_dir/data/'
all_data_folders = sorted([i for i in os.listdir(all_data_dir) if 'ks25' in i])
print(all_data_folders[selected_session])
root_dir = os.path.join(all_data_dir,all_data_folders[selected_session])
spkT = np.load(os.path.join(root_dir,'spkT_task.npy'))


#This array is the same shape as spkT but shows which cluster each of the spikes in spkT belongs to
spkC = np.load(os.path.join(root_dir,'spkC_task.npy'))

#This is basically a big table (you can open it in excel) which contains
#relevant information about each time the animal poked one of the ports
task_event_df = pd.read_csv(os.path.join(root_dir,'task_event_table.csv'),index_col=0)

#
response_table = np.load(os.path.join(root_dir,'neuron_response_table.npy'))
#alternatively to change the time window


#not all cluster in spkC correspond to single units. Single units is an array of the clusters that are single units
single_units = np.load(os.path.join(root_dir,'single_units.npy'))

2021-07-27_39955_ks25


In [397]:
graph_type0 = task_event_df.loc[task_event_df['task_nr']==0]['graph_type'].values[0]
graph_type1 = task_event_df.loc[task_event_df['task_nr']==1]['graph_type'].values[0]

seq0 = np.array(eval(task_event_df.loc[task_event_df['task_nr']==0]['current_sequence'].values[0]))
seq1 = np.array(eval(task_event_df.loc[task_event_df['task_nr']==1]['current_sequence'].values[0]))
print(graph_type0,graph_type1)

loop loop


In [398]:
def get_unique_transitions(seq0: np.ndarray,seq1: np.ndarray):
    
    test_seq1 = []
    for ix,i in enumerate(seq1):
        #print(seq0[(np.where(seq0==i)[0]-1)%9],seq0[(np.where(seq0==i)[0])%9],seq1[(ix-1)%9],seq1[(ix)%9])
        if seq0[(np.where(seq0==i)[0]-1)%9]!=seq1[(ix-1)%9]:
            test_seq1.append(1)
        else:
            #print(seq1[ix])
            test_seq1.append(0)

    test_seq1 = np.array(test_seq1)
    return test_seq1

In [399]:
def load_data(selected_session):
    all_data_dir = '/Users/yves/Desktop/retreat_data_dir/data/'
    all_data_folders = sorted([i for i in os.listdir(all_data_dir) if 'ks25' in i])
    root_dir = os.path.join(all_data_dir,all_data_folders[selected_session])
    spkT = np.load(os.path.join(root_dir,'spkT_task.npy'))


    #This array is the same shape as spkT but shows which cluster each of the spikes in spkT belongs to
    spkC = np.load(os.path.join(root_dir,'spkC_task.npy'))

    #This is basically a big table (you can open it in excel) which contains
    #relevant information about each time the animal poked one of the ports
    task_event_df = pd.read_csv(os.path.join(root_dir,'task_event_table.csv'),index_col=0)

    #
    response_table = np.load(os.path.join(root_dir,'neuron_response_table.npy'))
    #alternatively to change the time window


    #not all cluster in spkC correspond to single units. Single units is an array of the clusters that are single units
    single_units = np.load(os.path.join(root_dir,'single_units.npy'))
    
    
    seq0 = np.array(eval(task_event_df.loc[task_event_df['task_nr']==0]['current_sequence'].values[0]))
    seq1 = np.array(eval(task_event_df.loc[task_event_df['task_nr']==1]['current_sequence'].values[0]))
    
    
    graph_type0 = task_event_df.loc[task_event_df['task_nr']==0]['graph_type'].values[0]
    graph_type1 = task_event_df.loc[task_event_df['task_nr']==0]['graph_type'].values[0]
    
    firing_rate_maps = get_task_responses(task_event_df,response_table)
    
    return firing_rate_maps, task_event_df, (seq0,seq1,graph_type0,graph_type1)

In [400]:
#np.unique(task_event_df['graph_type'])

In [401]:
seq0 = np.array(eval(task_event_df.loc[task_event_df['task_nr']==0]['current_sequence'].values[0]))
seq1 = np.array(eval(task_event_df.loc[task_event_df['task_nr']==1]['current_sequence'].values[0]))

In [402]:
graph_type0 = task_event_df.loc[task_event_df['task_nr']==0]['graph_type'].values[0]
graph_type1 = task_event_df.loc[task_event_df['task_nr']==0]['graph_type'].values[0]

seq0 = np.array(eval(task_event_df.loc[task_event_df['task_nr']==0]['current_sequence'].values[0]))
seq1 = np.array(eval(task_event_df.loc[task_event_df['task_nr']==1]['current_sequence'].values[0]))

In [403]:
def get_task_responses(task_event_df,response_table):
    """ 
    Use the columns of the task_event_df to filter neural activity. 
    In this example build separate firing rate maps for each of the
    tasks, selecting only trials where subjects poked the correct poke.
    """
    
    
    n_neurons = response_table.shape[1]
    n_ports = 9
    n_tasks = 2
    n_direction = 2
    
    #set variables to nan to not confuse missing data for no responses
    firing_rate_maps = np.zeros([n_neurons,n_ports,n_tasks,n_direction]) + np.nan
    
    #for each task
    for task in [0,1]:
        
        for port in range(n_ports):  #for each port
            
            for dix,direction in enumerate(np.unique(task_event_df['direction'].values)):

                #Select indices of pokes where...
                table_index = task_event_df.loc[(task_event_df['task_nr']==task) &  #task_nr was task
                                                (task_event_df['correct']==True) &  #the poke was to the correct port
                                                (task_event_df['port']==port) &       #the port poked was port
                                                (task_event_df['direction']==direction)
                                               ].index           
                #print(len(table_index))
                #get the average
                firing_rate_maps[:,int(port),int(task),dix] = np.nanmean(response_table[table_index],axis=0)
    return firing_rate_maps
                                         

In [404]:
firing_rate_maps = get_task_responses(task_event_df,response_table)

In [405]:
n_neurons, n_ports, n_tasks, n_directions = firing_rate_maps.shape

In [406]:
#spatial_map  = np.nanmean(firing_rate_maps,axis=(2,3))

In [407]:
#mds_frm = firing_rate_maps - spatial_map[:,:,None,None]

In [408]:
from typing import List
def fit_svm_classify_task(X_train: np.ndarray, y_train: np.ndarray,
                          X_test: np.ndarray, y_test: np.ndarray) -> float:
    """
    
    """

    svm = SVC(kernel='linear')
    svm.fit(X_train.T,y_train)
    return np.mean(svm.predict(X_test.T)==y_test)

In [409]:
def create_task_train_test(task_X: np.ndarray,
                           train_fraction: float=0.8, rnd_seed=None
                           )-> Tuple[np.ndarray, np.ndarray]:
    
    
    n_ports1 = task_X.shape[1]
        
    np.random.seed(rnd_seed)
    
    perm_task1 = np.random.permutation(np.arange(n_ports1))
    
    n_train1 = int(np.floor(train_fraction*n_ports1))
    
    train_X, test_X = task_X[:,perm_task1[:n_train1]], task_X[:,perm_task1[n_train1:]]
    
    return train_X, test_X
    

In [410]:
np.random.seed(None)

In [411]:
frm_task.shape

(262, 9, 2)

In [412]:
#
task1_dir1 = np.logical_not(np.isnan(frm_task1[0,:,0]))
task1_dir2 = np.logical_not(np.isnan(frm_task1[0,:,1]))

In [413]:
# these firing rate maps are sorted by port NOT BY TASK
frm_task1 = firing_rate_maps[:,:,1]
frm_task2 = firing_rate_maps[:,:,0]

In [414]:
def get_valid_pokes_index_by_direction(frm_task: np.ndarray) -> Tuple[np.ndarray,np.ndarray]:

    #
    task_dir1 = np.where(np.logical_not(np.isnan(np.sum(frm_task,axis=0)[:,0])))[0]
    task_dir2 = np.where(np.logical_not(np.isnan(np.sum(frm_task,axis=0)[:,1])))[0]

    
    return task_dir1, task_dir2

In [415]:
task1_dir1, task1_dir2 = get_valid_pokes_index_by_direction(frm_task1)
task2_dir1, task2_dir2 = get_valid_pokes_index_by_direction(frm_task2)

In [416]:
def get_task_X(frm_task: np.ndarray, task_dir1: np.ndarray,
               task_dir2: np.ndarray) -> np.ndarray:
    
    task_X = np.hstack([frm_task[:,task_dir1,0],
                        frm_task[:,task_dir2,1]  
    ])


    return task_X

In [417]:
task1_dir2

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [418]:
task1_dir1, task1_dir2 = get_valid_pokes_index_by_direction(frm_task1)
task2_dir1, task2_dir2 = get_valid_pokes_index_by_direction(frm_task2)


task1_X = get_task_X(frm_task1, task1_dir1, task1_dir2)
task2_X = get_task_X(frm_task2, task2_dir1, task2_dir2)


train1_X, test1_X = create_task_train_test(task1_X)
train2_X, test2_X = create_task_train_test(task2_X)

In [419]:
task1_X = get_task_X(frm_task1, task1_dir1, task1_dir2)
task2_X = get_task_X(frm_task2, task2_dir1, task2_dir2)

In [420]:
np.all(np.isfinite(task2_X))

True

In [421]:
def create_task_train_test(task_X: np.ndarray,
                           train_fraction: float=0.8, rnd_seed=None
                           )-> Tuple[np.ndarray, np.ndarray]:
    
    
    n_ports1 = task_X.shape[1]
        
    np.random.seed(rnd_seed)
    
    perm_task1 = np.random.permutation(np.arange(n_ports1))
    
    n_train1 = int(np.floor(train_fraction*n_ports1))
    
    train_X, test_X = task_X[:,perm_task1[:n_train1]], task_X[:,perm_task1[n_train1:]]
    
    # check that order correctly determined
    assert(np.allclose(train_X[:,0],task_X[:,perm_task1[0]]))
    
    # and that no funky duplication is going on
    assert(np.sum(train_X,axis=0).shape[0]==len(np.unique(np.sum(train_X,axis=0))))
    
    return train_X, test_X
    

In [422]:
len(np.unique(np.sum(train_X,axis=0)))

12

In [423]:
train_X.shape

(262, 12)

In [424]:
np.all(np.isfinite(train2_X))

True

In [425]:
def build_features_and_labels(X1: np.ndarray, X2: np.ndarray,
                              ) -> Tuple[np.ndarray, np.ndarray]:
    """X1.shape=(n_features, n_samples)"""
    X = np.hstack([X1,X2])
    y = np.concatenate([np.ones(X1.shape[1]),
                              np.zeros([X2.shape[1]])
                             ])
    return X, y

In [435]:
import warnings
warnings.filterwarnings('ignore')

In [436]:
mu = []
    
for session_ix in range(4,8):
    sys.stdout.write("\rsession_index:{}".format(session_ix))
    sys.stdout.flush()
    tmp_mu = []
    for _ in range(50):

        firing_rate_maps,_,_ = load_data(session_ix)
        frm_task1 = firing_rate_maps[:,:,1]
        frm_task2 = firing_rate_maps[:,:,0]

        task1_dir1, task1_dir2 = get_valid_pokes_index_by_direction(frm_task1)
        task2_dir1, task2_dir2 = get_valid_pokes_index_by_direction(frm_task2)


        task1_X = get_task_X(frm_task1, task1_dir1, task1_dir2)
        task2_X = get_task_X(frm_task2, task2_dir1, task2_dir2)


        train1_X, test1_X = create_task_train_test(task1_X,rnd_seed=None)
        train2_X, test2_X = create_task_train_test(task2_X,rnd_seed=None)

        X_train, y_train = build_features_and_labels(train1_X,train2_X)
        X_test, y_test = build_features_and_labels(test1_X,test2_X)


        tmp_mu.append(fit_svm_classify_task(X_train,y_train,X_test,y_test))
#                                        ))
    mu.append(tmp_mu.copy())

session_index:7

In [437]:
np.mean(mu)

0.760625

In [438]:
np.mean(mu)

0.760625

In [282]:
np.random.shuffle(y_test)

In [270]:
np.all(np.isfinite(X_train))

True

In [271]:
fit_svm_classify_task(X_train,y_train,X_test,y_test)

1.0

In [329]:
valid_ix = np.where(~np.isnan(np.sum(firing_rate_maps,axis=(0,2,3))))[0]

real_acoss_task_fit = np.abs(get_svm_fit_across_task(frm_task,frm_task2,
                                                     valid_ix,
                                                     overlapping_ports,
                                                     do_shuffle=False)
                             -.5)
permutation_fits = [np.abs(get_svm_fit_across_task(frm_task,frm_task2,
                                            valid_ix,
                                            overlapping_ports,
                                            do_shuffle=True) -.5)
                    for _ in range(500)]
                          
#mean_perm_fits = [np.mean(i) for i in permutation_fits]

[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]


In [330]:
stt.percentileofscore(permutation_fits,real_acoss_task_fit)

28.1

In [331]:
real_acoss_task_fit

0.0

# Now do the same but separating by direction